Data Fetching from Yahoo Finacial API<br>
Data preprocessing and encoding<br>
Model Define<br>

<h1>Model Inputs:</h1>
<li>Baisc Securites Information</li>
<li>Historical Stock value</li>
<li>News (National News and World News Headlines)</li>
<h1>Model Output</h1>
<p>Stock Prediction for 1 day in 1 hour interval
[ Monday through Friday from 9:15 AM to 3:30 PM India Standard Time (GMT+05:30) ]

Shape of Output: [6, 7]</p>

Fetch List of Securities available in NSE from list

In [ ]:
!pip install gnewsclient
!pip install transformers
!pip install tqdm boto3 requests regex sentencepiece sacremoses

In [ ]:
# /content/drive/MyDrive/mtech_project/stock_data/Securities_available_for_Equity_segment.csv
# /content/drive/MyDrive/mtech_project/stock_data/Securities_available_for_trading_in_SME.csv

# get list of symbols available
import pandas as pd

data1 = pd.read_csv(r"/content/drive/MyDrive/mtech_project/stock_data/Securities_available_for_Equity_segment.csv")
data2 = pd.read_csv(r"/content/drive/MyDrive/mtech_project/stock_data/Securities_available_for_trading_in_SME.csv")

#print(data1['SYMBOL'])

#ata1 = pd.concat([data1, data2], axis=0)
eq_symbols = [symbol+".NS" for symbol in data1['SYMBOL']]
sme_symbols = [symbol+".NS" for symbol in data2['SYMBOL']]

eq_isin = [no for no in data1[' ISIN_NUMBER']]
sme_isin = [no for no in data2['ISIN_NUMBER']]

print(len(eq_symbols), len(eq_isin), len(sme_isin), len(sme_symbols))
# verifying both are correct
assert len(eq_symbols) == len(eq_isin), "Equity symbol and ISIN number should be equal."
assert len(sme_symbols) == len(sme_isin), "SME symbol and ISIN number should be equal."

tuples = zip(eq_symbols, eq_isin)
eq_seg = dict(tuples) # Equity segments
tuples = zip(sme_symbols, sme_isin)
sme_seg = dict(tuples) # Small and medium scale segments

print(eq_seg)
print(sme_seg)

1895 1895 192 192
{'20MICRONS.NS': 'INE144J01027', '21STCENMGM.NS': 'INE253B01015', '360ONE.NS': 'INE466L01038', '3IINFOLTD.NS': 'INE748C01038', '3MINDIA.NS': 'INE470A01017', '3PLAND.NS': 'INE105C01023', '5PAISA.NS': 'INE618L01018', '63MOONS.NS': 'INE111B01023', 'A2ZINFRA.NS': 'INE619I01012', 'AAATECH.NS': 'INE0D0U01013', 'AAKASH.NS': 'INE087Z01024', 'AAREYDRUGS.NS': 'INE198H01019', 'AARON.NS': 'INE721Z01010', 'AARTECH.NS': 'INE01C001018', 'AARTIDRUGS.NS': 'INE767A01016', 'AARTIIND.NS': 'INE769A01020', 'AARTIPHARM.NS': 'INE0LRU01027', 'AARTISURF.NS': 'INE09EO01013', 'AARVEEDEN.NS': 'INE273D01019', 'AARVI.NS': 'INE754X01016', 'AAVAS.NS': 'INE216P01012', 'ABAN.NS': 'INE421A01028', 'ABB.NS': 'INE117A01022', 'ABBOTINDIA.NS': 'INE358A01014', 'ABCAPITAL.NS': 'INE674K01013', 'ABFRL.NS': 'INE647O01011', 'ABMINTLLTD.NS': 'INE251C01025', 'ABSLAMC.NS': 'INE404A01024', 'ACC.NS': 'INE012A01025', 'ACCELYA.NS': 'INE793A01012', 'ACCURACY.NS': 'INE648Z01023', 'ACE.NS': 'INE731H01025', 'ACEINTEG.NS': 'I

In [14]:
# Basis information by ticker name

import yfinance as yf

def get_fundamental_info(ticker_name):
  """Divide the input dictinary into multiple seq with specified length.

  Args:
    ticker_name: A SYMBOL.

  Returns:
    info: A dict with basic information about the given symbol.
    hist: historical market data of period 1 day and interval of 1 min
  """
  # get basic data
  ticker = yf.Ticker(ticker_name)

  # get historical market data
  hist = ticker.history(period="5d", interval="1m")

  return hist

In [15]:
get_fundamental_info('20MICRONS.NS')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-11-24 09:15:00+05:30,180.500000,182.000000,180.500000,182.000000,0,0.0,0.0
2023-11-24 09:16:00+05:30,182.000000,182.000000,181.500000,181.500000,954,0.0,0.0
2023-11-24 09:17:00+05:30,183.000000,183.000000,183.000000,183.000000,1800,0.0,0.0
2023-11-24 09:18:00+05:30,183.199997,183.199997,183.100006,183.199997,4790,0.0,0.0
2023-11-24 09:19:00+05:30,184.699997,184.699997,183.199997,183.199997,311,0.0,0.0
...,...,...,...,...,...,...,...
2023-12-01 15:25:00+05:30,175.250000,175.800003,175.250000,175.800003,1634,0.0,0.0
2023-12-01 15:26:00+05:30,175.800003,175.800003,175.250000,175.250000,873,0.0,0.0
2023-12-01 15:27:00+05:30,175.300003,176.000000,175.250000,176.000000,499,0.0,0.0


In [ ]:
# Get market news and parse it
from gnewsclient import gnewsclient

def get_national_news():
  client = gnewsclient.NewsClient(language='en', location='india', topic='Business', max_results=100)
  news = client.get_news() # list containes dict
  national_news = list()
  for title in news:
    national_news.append(title['title']+'. ')
  return national_news

news = get_national_news()
print("No of news: ", len(news), ", News: At Index[0] ", news[0], ", At Index[", len(news), "] ", news[len(news)-1])

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


No of news:  70 , News: At Index[0]  Nifty 50, Sensex today: What to expect from stock market indices in trade on October 12 | Mint - Mint.  , At Index[ 70 ]  Urban Co forays into consumer appliances | Mint - Mint. 


In [ ]:
# Get market news and parse it
def get_world_news_by_isin(isin):
  news = yf.utils.get_news_by_isin(isin)
  world_news = list()
  for title in news:
    world_news.append(title['title']+'. ')
  return world_news

news = get_world_news_by_isin('INE253B01015')
print("No of news: ", len(news), ", News: At Index[0] ", news[0], ", At Index[", len(news), "] ", news[len(news)-1])

No of news:  8 , News: At Index[0]  Wells Fargo rolls out financial planning tool to almost 70 million customers.  , At Index[ 8 ]  United States T-Commerce (Buying Through TV) Market Study 2023 Featuring Apple, Amazon, Google, Samsung, & Android. 


In [ ]:
# Data pre-processing
def divide_dict_into_seq1(dict1):
  """Divide the input dictinary into multiple seq with specified length.

  Args:
    dict1: A dictionary.

  Returns:
    A list with divided seq.
  """
  #print(dict1)
  max_seq_len = 800
  seq = ""
  tot_seq = list()
  for key, value in dict1.items():
    length = len(key)+len(str(value))

    if max_seq_len >= length:
      seq += str(key)+" is "+str(value) + ". "
      max_seq_len -= length
    else:
      max_seq_len = 800
      tot_seq.append(seq)
      seq = ""
      seq += str(key)+" is "+str(value) + ". "
      max_seq_len -= length
  del seq
  return tot_seq

def divide_dict_into_seq(dict1):
  """Divide the input dictinary into multiple seq with specified length.

  Args:
    dict1: A dictionary.

  Returns:
    A list with divided seq.
  """
  #print(dict1)
  tot_seq = list()
  for key, value in dict1.items():
    tot_seq.append(str(key)+" is "+str(value) + ". ")
  return tot_seq


def string_into_seq(news_list):
  """Divide the input news list into multiple seq with specified length.

  Args:
    dict1: A dictionary.

  Returns:
    A list with divided seq.
  """
  max_seq_len = 800
  seq = ""
  tot_seq = list()
  for news in news_list:
    length = len(news)

    if max_seq_len >= length:
      seq += news+". "
      max_seq_len -= length
    else:
      max_seq_len = 800
      tot_seq.append(seq)
      seq = ""
      seq += news+". "
      max_seq_len -= length
  del seq
  return tot_seq

In [ ]:
# Dividing table into input seq
info, hist, news = get_fundamental_info('21STCENMGM.NS')
val = hist.iloc[0].to_string()

def clean_text(text):
  text = text.replace(" ", "")
  text = text.replace('Open', 'Opening is ')
  text = text.replace('\n', ', ')
  text = text.replace('High', ' high is ')
  text = text.replace('Low', ' low is ')
  text = text.replace('Close', ' closing is ')
  text = text.replace('Volume', ' volume traded is ')
  text = text.replace('Dividends', ' dividends are ')
  text = text.replace('StockSplits', ' stock splits are ')
  return text.strip()

def table_to_seq(hist):
  """Divide the input news list into multiple seq with specified length.

  Args:
    hist: A dataframe.

  Returns:
    A list with divided seq.
  """
  seq_li = list()
  no_of_row = hist.shape[0]
  for i in range(no_of_row):
    seq_li.append(clean_text(hist.iloc[i].to_string()) + ", at " + str(hist.iloc[i].name)+". ")
  return seq_li

news = table_to_seq(hist)
print("No of rows: ", len(news), ", values: At Index[0] ", news[0], ", At Index[", len(news), "] ", news[len(news)-1])

No of rows:  16 , values: At Index[0]  Opening is 18.799999,  high is 18.799999,  low is 18.799999,  closing is 18.799999,  volume traded is 0.000000,  dividends are 0.000000,  stock splits are 0.000000, at 2023-10-12 09:23:00+05:30.  , At Index[ 16 ]  Opening is 18.85,  high is 18.85,  low is 18.85,  closing is 18.85,  volume traded is 85.00,  dividends are 0.00,  stock splits are 0.00, at 2023-10-12 15:27:00+05:30. 


In [ ]:
# Tokenization
# Gather all data
def get_data(sec_name, isin):
  info, hist = get_fundamental_info(sec_name)
  n_news = get_national_news()
  w_news = get_world_news_by_isin(isin)

  input = list()
  for data in divide_dict_into_seq(info):
    input.append(data)

  for data in n_news:
    input.append(data)

  for data in w_news:
    input.append(data)

  for data in table_to_seq(hist):
    input.append(data)

  print(len(input))
  print(type(input[0]))
  return input

189
<class 'str'>


In [ ]:
# Tokenzation
"""
input_data = {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "token_type_ids": segment_ids,
}

input_tensors = {
    k: torch.tensor([v]) for k, v in input_data.items()
}

with torch.no_grad():
    logits = model(**input_tensors)[0]

predicted_class = torch.argmax(logits, dim=1).item()
"""
import torch
from torch import nn

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

def tokenize(input):
  indexed_tokens = list()
  len_counter = 512
  seq_part = list()
  for data in input:
    for token in tokenizer.encode(data):
      if len_counter == 0:
        indexed_tokens.append(seq_part)
        seq_part = list()
        len_counter = 511
        seq_part.append(token)
      else:
        seq_part.append(token)
        len_counter -= 1

  tokens_tensor = torch.tensor(indexed_tokens)
  print('Input Tensor Size',tokens_tensor.shape)
  return tokens_tensor

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [ ]:

class Padding(nn.Module):
    def forward(self, x):
        # Output shape (batch, features, hidden))
        # batch size 16
        b_size = 16 - x.size()[0]
        padding_t = torch.zeros(b_size, 512*1024)
        if b_size < 0:
          print('Batch size is more than specified value 16.')
          raise Exception("Batch size is more than specified value 16.")
        x = torch.cat((x, padding_t), dim=0)
        return x

In [ ]:
# Model Input and Check

model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

# appending linear layer at the end BERT
flatten_layer = torch.nn.Flatten(start_dim = 1)
padding_layer = Padding()
linear_layer = torch.nn.Linear(16*512*1024, 42)
flatten_layer1 = torch.nn.Flatten(start_dim = 0)

model.add_module('flat_layer1', flatten_layer)
model.add_module('pad_layer', padding_layer)
model.add_module('lin_layer', linear_layer)
model.add_module('flat_layer2', flatten_layer1)



with torch.no_grad():
    output = model(tokens_tensor)

#print(output[0])
print(output[0].size()) # output from last layer
#print(output[1])
print(output[1].size())

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


Input Tensor Size torch.Size([9, 512])
torch.Size([9, 512, 1024])
torch.Size([9, 1024])


In [ ]:
# Prediction and Training logic
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Criterian for GA and Adam Implimentation
model.train()

# Obtaine Data

input = get_data(sec_name, isin)
tokens_tensor = tokenize(input)
output = model(tokens_tensor)


In [ ]:
# Optimization